In [22]:
import numpy as np
import pandas as pd
from scipy import stats
from bs4 import BeautifulSoup
import requests
import time

In [23]:
start_year = 2010
end_year = 2019
df_combined_matches = pd.DataFrame()

for year in range(start_year, end_year + 1):
    url = "https://www.footywire.com/afl/footy/ft_match_list?year={}".format(year)
    res = res = requests.get(url)
    soup = BeautifulSoup(res.text, features='lxml')
    tables = soup.select(".data")
    
    years = []
    dates = []
    teams = []
    venues = []
    crowds = []
    results = []
    match_ids = []
    disposals = []
    goals = []

    counter = 0

    for cell in tables:
        if counter == 0:
            years.append(year)
            dates.append(cell.text)
        elif counter == 1:
            teams.append(cell.text)
        elif counter == 2:
            venues.append(cell.text)
        elif counter == 3:
            crowds.append(cell.text)
        elif counter == 4:
            results.append(cell.text)
            for a in cell.find_all('a', href=True):
                match_ids.append(a['href'])
        elif counter == 5:
            disposals.append(cell.text)
        elif counter == 6:
            goals.append(cell.text)
        counter += 1
        if counter == 7:
            counter = 0

    df_footywire_matches = pd.DataFrame(
        list(
            zip(
                years,
                dates,
                teams,
                venues,
                crowds,
                results,
                match_ids,
                disposals,
                goals,
            )
        ),
        columns=[
            'year',
            'date',
            'teams',
            'venue',
            'crowd',
            'results',
            'match_id',
            'disposals',
            'goals'
        ])

    split_teams = df_footywire_matches["teams"].str.split("\nv", n = 1, expand = True) 
    df_footywire_matches["home_team"]= split_teams[0] 
    df_footywire_matches["away_team"]= split_teams[1] 
    split_results = df_footywire_matches["results"].str.split("-", n = 1, expand = True) 
    df_footywire_matches["home_team_score"]= split_results[0] 
    df_footywire_matches["away_team_score"]= split_results[1] 
    df_footywire_matches.drop(columns =["teams", "results", "disposals", "goals"], inplace = True) 

    df_footywire_matches['home_team'] = df_footywire_matches['home_team'].str.replace('\n', '')
    df_footywire_matches['away_team'] = df_footywire_matches['away_team'].str.replace('\n', '')
    df_footywire_matches['match_id'] = df_footywire_matches['match_id'].str.replace('ft_match_statistics\?mid=', '')
    df_footywire_matches['match_id'] = df_footywire_matches['match_id'].astype(int)
    df_combined_matches = pd.concat([df_combined_matches ,df_footywire_matches])

df_combined_matches

,year,date,venue,crowd,match_id,home_team,away_team,home_team_score,away_team_score
0,2010,Thu 25 Mar 7:10pm,MCG,72010,5089,Richmond,Carlton,64,120
1,2010,Fri 26 Mar 7:40pm,MCG,57772,5090,Geelong,Essendon,125,94
2,2010,Sat 27 Mar 2:10pm,MCG,45615,5091,Melbourne,Hawthorn,61,117
3,2010,Sat 27 Mar 7:10pm,ANZ Stadium,31330,5093,Sydney,St Kilda,88,96
4,2010,Sat 27 Mar 7:30pm,Gabba,29201,5092,Brisbane,West Coast,114,82
...,...,...,...,...,...,...,...,...,...
202,2019,Sat 17 Aug 6:10pm,Optus Stadium,47833,9923,Fremantle,Essendon,55,87
203,2019,Sat 17 Aug 7:25pm,Marvel Stadium,17063,9924,North Melbourne,Port Adelaide,144,58
204,2019,Sun 18 Aug 1:10pm,MCG,57415,9925,Richmond,West Coast,88,82
205,2019,Sun 18 Aug 3:20pm,GIANTS Stadium,10139,9926,GWS,Western Bulldogs,65,126


In [24]:
timestr = time.strftime("%Y%m%d-%H%M%S")
df_combined_matches.to_csv("Outputs/footywire_team_matches_{}.csv".format(timestr))